# [E-09] Movielens 영화 추천 실습

## 1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2. 데이터 분석

### 2.1 ratings에 있는 유니크한 영화 개수

In [6]:
ratings['movie_id'].nunique()

3628

### 2.2 ratings에 있는 유니크한 사용자 수

In [7]:
ratings['user_id'].nunique()

6039

### 2.2 가장 인기 있는 영화 30개(인기순)

In [8]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
best_movie = movie_count.sort_values(ascending=False).head(30)

In [9]:
best_movie = best_movie.reset_index()
pd.merge(best_movie, movies)

,movie_id,user_id,title,genre
0,2858,3211,American Beauty (1999),Comedy|Drama
1,260,2910,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
2,1196,2885,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
3,1210,2716,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
4,2028,2561,Saving Private Ryan (1998),Action|Drama|War
5,589,2509,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
6,593,2498,"Silence of the Lambs, The (1991)",Drama|Thriller
7,1198,2473,Raiders of the Lost Ark (1981),Action|Adventure
8,1270,2460,Back to the Future (1985),Comedy|Sci-Fi
9,2571,2434,"Matrix, The (1999)",Action|Sci-Fi|Thriller


## 3 데이터 전처리

### 3.1 데이터 합치기

In [10]:
data = pd.merge(ratings, movies)
data = data.drop(['movie_id','timestamp','genre'], axis=1)
data

,user_id,count,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,2,5,One Flew Over the Cuckoo's Nest (1975)
2,12,4,One Flew Over the Cuckoo's Nest (1975)
3,15,4,One Flew Over the Cuckoo's Nest (1975)
4,17,5,One Flew Over the Cuckoo's Nest (1975)
...,...,...,...
836473,5851,5,One Little Indian (1973)
836474,5854,4,Slaughterhouse (1987)
836475,5854,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,4,"Five Wives, Three Secretaries and Me (1998)"


### 3.2 내가 선호하는 영화

In [11]:
# 넣다보니 다 로맨스군요,, 이거 다 재미있습니다 
my_favorite = ['Toy Story (1995)', 'Pretty Woman (1990)', 'Notting Hill (1999)', 'Breakfast at Tiffany\'s (1961)', 'Ghost (1990)']

# 'mink'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['mink']*5, 'title': my_favorite, 'count':[5]*5})

if not data.isin({'user_id':['mink']})['user_id'].any():    # user_id에 'mink'이라는 데이터가 없다면
    data = data.append(my_playlist)                          # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.


,user_id,count,title
836473,5851,5,One Little Indian (1973)
836474,5854,4,Slaughterhouse (1987)
836475,5854,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,5,Identification of a Woman (Identificazione di ...
0,mink,5,Toy Story (1995)
1,mink,5,Pretty Woman (1990)
2,mink,5,Notting Hill (1999)
3,mink,5,Breakfast at Tiffany's (1961)
4,mink,5,Ghost (1990)


In [12]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [13]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['mink'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(movie_to_idx['Pretty Woman (1990)'])

6039
336


In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,count,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,40
1,6039,5,336
2,6039,5,1151
3,6039,5,566


## 4. CSR(Compressed Sparse Row) matrix

### 4.1 Sparse Matrix (희소 행렬)
희소행렬 : 대부분의 값이 0으로 채워진 행렬 (<-> 밀집 행렬(Dense Matrix)
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcFkDRL%2FbtqBUAYOqiE%2FMTeuu3LmXbaHeKce26OXz1%2Fimg.png)
  
  
### 4.2 COO (Coordinate : 좌표) 형식  
0이 아닌 데이터만 별도의 배열에 저장하고, 그데이터가 가리키는 행과 위치를 별도의 배열에 저장하는 방식


### 4.3 CSR (Compressed Sparse Row) 형식
가로의 순서대로 재정렬하는 방법으로 행에 관여하여 정리 압축하는 방식


그 외의 희소행렬의 자료구조 저장법 https://ko.wikipedia.org/wiki/%ED%9D%AC%EC%86%8C%ED%96%89%EB%A0%AC

In [15]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), shape=(num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

(user_id + 나) x (영화 수) 로 잘 들어가 있다.

## 5. 학습 시키기 (Training)

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [20]:
mink, PrettyWoman = user_to_idx['mink'], movie_to_idx['Pretty Woman (1990)']
mink_vector, PrettyWoman_vector = als_model.user_factors[mink], als_model.item_factors[PrettyWoman]

In [21]:
mink_vector

array([ 3.87676060e-01,  9.29304399e-03,  6.23047888e-01,  1.38213247e-01,
        5.27078927e-01, -7.45494485e-01, -3.00575376e-01,  3.46557423e-02,
       -7.44575500e-01,  8.06580722e-01,  1.65618718e-01,  7.04797924e-01,
       -9.58043993e-01, -8.17972422e-03,  3.15427065e-01, -3.01445693e-01,
       -6.89876020e-01, -1.09789439e-01,  4.20397252e-01, -3.68407637e-01,
        2.80022174e-01, -2.22454235e-01,  2.33665347e-01,  4.78329420e-01,
        1.49313495e-01, -1.18302129e-01, -2.61264712e-01, -1.52809203e-01,
       -6.15323603e-01,  1.40007789e-04,  8.42610121e-01, -5.87403119e-01,
       -7.85424411e-01,  1.00108366e-02,  3.60465527e-01,  6.05934739e-01,
        7.53865182e-01, -1.03922594e+00,  3.51688087e-01, -4.66822326e-01,
       -2.60634333e-01, -3.53901565e-01, -2.47884050e-01, -3.82355422e-01,
       -2.49507964e-01, -1.13587596e-01,  6.24628365e-01, -1.62796468e-01,
        1.92692831e-01,  1.04386711e+00,  8.28361452e-01, -4.35304970e-01,
       -2.50178456e-01, -

In [22]:
PrettyWoman_vector

array([ 0.01454473,  0.00270258,  0.02041643,  0.02745439,  0.00144375,
        0.00067697, -0.00202728, -0.00329987, -0.00759036,  0.00728766,
       -0.00520682,  0.02193808, -0.00667134,  0.010587  ,  0.01410219,
        0.00775932, -0.00700677,  0.01343374,  0.01421065, -0.00271833,
       -0.00185864, -0.01371401,  0.00733077,  0.00602728, -0.00676426,
       -0.00019197, -0.01457777,  0.01799463, -0.00874509,  0.00886609,
        0.00972143, -0.01809237,  0.00397536,  0.0101153 ,  0.01104434,
        0.03349006,  0.04218306, -0.00652647,  0.01354336,  0.00208092,
        0.02608089, -0.0049196 ,  0.00647196, -0.00255887,  0.00890101,
        0.00112414,  0.00550984,  0.00193407,  0.02752604,  0.04771416,
        0.0275705 , -0.01040633,  0.00742405,  0.0132195 ,  0.00949218,
        0.01296193,  0.00318249,  0.04052561,  0.02267224, -0.01423843,
       -0.00710736,  0.01398131,  0.02536487,  0.01213174, -0.00138095,
        0.02195136, -0.01612398,  0.00609826,  0.00039484,  0.00

### 내가 선호하는 5가지 영화 와 나의 선호도 

In [23]:
np.dot(mink_vector, PrettyWoman_vector)

0.4895228

### 내가 선호하는 다른 영화 와 나의 선호도

In [24]:
BaB = movie_to_idx['Beauty and the Beast (1991)']
BaB_vector = als_model.item_factors[BaB]
np.dot(mink_vector, BaB_vector)

0.08806912

내가 좋아하는 영화 장르가 하나 빼고 (하나만 애니메이션)이고 다 로맨스라서 실제로 좋아하는 미녀와 야수와는 많이 멀어 보인다,,ㅜ 미녀와 야수도 좋아하는 영화인데,, 확실히 내 선호 영화랑 좀 다른 느낌이하다.

## 6. 영화 추천

In [25]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_movie(title: str):
    movie_id = movie_to_idx[title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [26]:
get_similar_movie('Pretty Woman (1990)')

['Pretty Woman (1990)',
 'Sleepless in Seattle (1993)',
 "My Best Friend's Wedding (1997)",
 'Ghost (1990)',
 "You've Got Mail (1998)",
 'While You Were Sleeping (1995)',
 'Notting Hill (1999)',
 'Truth About Cats & Dogs, The (1996)',
 'Sabrina (1995)',
 'Mrs. Doubtfire (1993)']

In [27]:
user = user_to_idx['mink']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
[idx_to_movie[i[0]] for i in movie_recommended]

['Sleepless in Seattle (1993)',
 'Toy Story 2 (1999)',
 "You've Got Mail (1998)",
 'Forrest Gump (1994)',
 "My Best Friend's Wedding (1997)",
 "Bug's Life, A (1998)",
 'Groundhog Day (1993)',
 'While You Were Sleeping (1995)',
 'Four Weddings and a Funeral (1994)',
 'Dave (1993)',
 'Babe (1995)',
 'Shakespeare in Love (1998)',
 'Sabrina (1995)',
 'Tin Cup (1996)',
 'Mrs. Doubtfire (1993)',
 'American President, The (1995)',
 'Truth About Cats & Dogs, The (1996)',
 'Clueless (1995)',
 'Birdcage, The (1996)',
 'Wedding Singer, The (1998)']

## 회고
나는 옛날 외국 영화와 노래를 좋아한다. 놀랍게도 추천해준 영화중에 실제로 좋아하고 본 영화들도 있어서 신기했다.  
이번에 추천 시스템에 대해 처음 접해 봤다.  
이번 모델도 잘 이해 못했는데 08/18 심야극장 주제가 추천 시스템이라서 참여 했다가,, 위의 추천 시스템은 아주 아주 쉬운 단계라는 걸 알게 됐고,, 네이버에서 만든 추천 시스템은 엄청 어려웠다. 여러가지 모델을 직접 실험해보고 토너먼트 형식으로 올라가면서 자동으로 하이퍼파라미터도 조정되는게 신기했다.  
처음으로 클라우드 주피터를 이용해봤는 데 익숙하지 않아서 불편하다.. 근데 데이터를 가져올때 바로 가져올 수 있는 점은 편리한 것 같다.

